<a href="https://colab.research.google.com/github/SEIGO55/intro_git/blob/main/Logistics%E5%9B%9E%E5%B8%B0%E3%81%A7%E4%BA%88%E6%83%B3%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

# Google ドライブをマウント
drive.mount('/content/drive')

files = ['2020JRAresults.csv', '2021JRAresults.csv', '2022JRAresults.csv']

# Google ドライブの特定のディレクトリのパスを指定
base_path = '/content/drive/My Drive/ADS競馬部/'

# 各ファイルを読み込み、一つのリストに格納
dfs = [pd.read_csv(base_path + file) for file in files]

# リスト内の全ての DataFrame を縦に連結
result_df = pd.concat(dfs, ignore_index=True)

grade = pd.read_csv(base_path + '重賞レースリスト.csv')

# 'race_name' をキーにして結合し、result_df の行数を変更しないように 'left' 結合を選択
merged_df = pd.merge(result_df, grade, on='race_name', how='left')

# 'Grade' 列に対して、NaN 値を '-' に置き換え
merged_df['Grade'].fillna('-', inplace=True)

# '開催日' 列を datetime 型に変換
merged_df['開催日'] = pd.to_datetime(merged_df['開催日'], format='%Y%m%d')

# 開催年を新しい列として追加（'開催日' 列の year 属性を取得）
merged_df['開催年'] = merged_df['開催日'].dt.year

# '開催年' 列をデータフレームの最初の列に移動
cols = ['開催年'] + [col for col in merged_df if col != '開催年']
df = merged_df[cols]

# '距離' 列から 'コース'、'距離'、'コースタイプ' の情報を正規表現で抽出
df[['コース', '距離2']] = df['距離'].str.extract(r'([ダ芝]+)(\d+m)')
df['コースタイプ'] = df['距離'].str.extract(r'\((.*?)\)')

# '(増減)'列の値を整形し、'NaN' 文字列を実際のNaNに変換し、NaNを0に置き換え、整数型に変換する
df['(増減)'] = df['(増減)'].str.replace('(', '').str.replace(')', '')
df['体重変化'] = df['(増減)'].replace('NaN', np.nan).fillna(0).astype(int)
# '着順' 列の値が整数の場合、かつ1から20までの範囲にある場合は削除しない
df = df[~df['着順'].apply(lambda x: isinstance(x, str) and (not x.isdigit() or int(x) < 1 or int(x) > 20))]
df['着順']=df['着順'].astype(int)

Mounted at /content/drive


<ipython-input-1-041488db2443>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['コース', '距離2']] = df['距離'].str.extract(r'([ダ芝]+)(\d+m)')
<ipython-input-1-041488db2443>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['コース', '距離2']] = df['距離'].str.extract(r'([ダ芝]+)(\d+m)')
<ipython-input-1-041488db2443>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [ ]:
new_col = ['着順', '枠', '馬番', '馬名', '斤量', '騎手',  '人気',
       '単勝オッズ', '性', '齢',  '馬体重', '体重変化','race_name','コース','距離2','コースタイプ','天候','馬場状態']
result=df[new_col]

# 'rank' 列を作成し、条件に基づいて値を設定
result['rank'] = result['着順'].apply(lambda x: x if str(x).isdigit() and int(x) in [1, 2, 3] else 4)


<ipython-input-2-33707d54e061>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rank'] = result['着順'].apply(lambda x: x if str(x).isdigit() and int(x) in [1, 2, 3] else 4)


In [ ]:
result.drop(['着順', '馬名'], axis=1, inplace=True)

<ipython-input-3-7ed6834b0de3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(['着順', '馬名'], axis=1, inplace=True)


In [ ]:
#ダミー変数化
result_d = pd.get_dummies(result)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = result_d.drop(['rank'], axis=1)
y = result_d['rank']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [ ]:
#アンダーサンプリング  それぞれのクラスの数を同数程度にするため
from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]
rus = RandomUnderSampler(sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1}, random_state=0)


X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [ ]:
#クラスごとに大体同数になっていることの確認
pd.Series(y_train_rus).value_counts()

1    7273
4    7273
3    7262
2    7255
Name: rank, dtype: int64

In [ ]:
#Model作ってスコアの計算
model = LogisticRegression()
model.fit(X_train_rus, y_train_rus)

print(model.score(X_train, y_train), model.score(X_test, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.55957966224294 0.5608063759962494
